<h1>Process Data</h1>
<h4>Haley Johnson</h4>

<p>Notebook to extract, combine and transform data</p>

In [2]:
import pandas as pd
import numpy as np
import os 

<h2>Load HTML File</h2>

In [30]:
html = pd.read_html("happiest_cities.html")

In [31]:
happiness_df = html[0]

In [34]:
happiness_df = happiness_df.rename(columns = {'City': 'Location'})

happiness_df['state'] = happiness_df['Location'].apply(lambda s: s.split(",")[-1])
happiness_df['city'] = happiness_df['Location'].apply(lambda s: s.split(",")[0])

happiness_df = happiness_df.drop(columns = ['Location'])

<h2>Load Transportation Data</h2>

In [13]:
locations_df = pd.read_csv("EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv")

In [15]:
# still deciding what to do with names

# locations_df.rename(columns = {"GEOID10": "2010_fips_code", "GEOID20" : "2018_fips_code", 
#                     "STATEFP": 'state_fips_code', "COUNTYFP": "county_fips_code", 
#                     "Ac_Total": 'total_acres', 'Ac_Land': 'land_acres', 'Ac_Water': 'water_acres', 
#                     "Ac_Unpr": "unprotected_acres", "CountHU": "total_housing_units", "HH": "households", 
#                     "P_WrkAge": 'percent_working_age', 'AutoOwn0': 'owns_no_cars', 
#                     'Pct_AO0': 'percent_owns_no_cars', "AutoOwn1": 'owns_one_car', "Pct_AO1": "percent_owns_one_car",
#                     "AutoOwn2p": "owns_two_plus_cars", "Pct_AO2": 'percent_owns_2_plus_cars'})

,OBJECTID,2010_fips_code,2018_fips_code,state_fips_code,county_fips_code,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.184697,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,2.978361e+05
1,2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.323221,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,4.849451e+05
2,3,4.811300e+11,4.811300e+11,48,113,7825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.314628,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,1.067059e+05
3,4,4.811300e+11,4.811300e+11,48,113,7824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.229821,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,4.818284e+05
4,5,4.811300e+11,4.811300e+11,48,113,7824,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.164863,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,6.876848e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220735,220736,7.803100e+11,7.803100e+11,78,30,961000,2,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,3414.446949,3.355857e+05
220736,220737,7.803100e+11,7.803100e+11,78,30,961000,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,2421.025608,2.924305e+05
220737,220738,7.803100e+11,7.803100e+11,78,30,961000,5,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,1955.909418,1.619395e+05
220738,220739,7.803100e+11,7.803100e+11,78,30,960700,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,10,1,4.000000,16896.768870,1.038966e+07


<h3>Get City and State Names</h3>

In [38]:
locations_df['metro_are_cities'] = locations_df['CBSA_Name'].apply(lambda s: str(s).split(",")[0])
locations_df['metro_area_states'] = locations_df['CBSA_Name'].apply(lambda s: str(s).split(",")[-1])

In [26]:
locations_df['ci'] = locations_df['metro_are_cities'].apply(lambda s: s.split("-"))
locations_df['metro_states']

In [37]:
locations_df['state'].unique()

array([' TX', 'nan', ' TX-AR', ' VA-NC', ' VA', ' DC-VA-MD-WV', ' VA-WV',
       ' TN-VA', ' WV-VA', ' AL', ' GA-AL', ' AL-GA', ' AK', ' AZ', ' AR',
       ' AR-OK', ' TN-MS-AR', ' CA', ' CO', ' CT', ' MA-CT',
       ' PA-NJ-DE-MD', ' MD-DE', ' DE', ' FL', ' GA', ' GA-SC', ' TN-GA',
       ' HI', ' ID', ' ID-WA', ' WY-ID', ' OR-ID', ' UT-ID', ' IL',
       ' IL-IN-WI', ' MO-IL', ' IA-IL', ' IL-MO', ' KY-IL', ' IA-IL-MO',
       ' IN', ' KY-IN', ' IN-MI', ' IN-KY', ' OH-KY-IN', ' IA', ' NE-IA',
       ' IA-NE-SD', ' MO-KS', ' KS', ' KY', ' WV-KY-OH', ' TN-KY', ' LA',
       ' MS-LA', ' ME', ' MD', ' MD-WV', ' MA', ' MA-NH', ' RI-MA', ' MI',
       ' WI-MI', ' MI-WI', ' MN-WI', ' MN', ' ND-MN', ' WI-MN', ' MS',
       ' MO', ' MT', ' NE', ' NV', ' NH', ' NH-VT', ' NY-NJ-PA', ' NJ',
       ' PA-NJ', ' NM', ' NY', ' NC', ' NC-SC', ' SC-NC', ' ND', ' OH',
       ' WV-OH', ' OH-PA', ' OK', ' OR', ' OR-WA', ' PA', ' SC', ' SD',
       ' TN', ' UT', ' VT', ' WA', ' WV', ' WI', ' WY', ' PR'],
 